<img src="https://imgur.com/3U3hI1u.png" width="100%" />

#### Boas vindas a sua segunda e última tarefa de Aprendizado por Reforço!

Neste exercício, você deverá implementar e comparar diferentes algoritmos de **Aprendizado por Reforço Profundo** utilizando a biblioteca _[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)_.

A _Stable Baselines_ é uma biblioteca de Aprendizado por Reforço que implementa diversos algoritmos de agentes, além de várias funcionalidades úteis para seu treinamento. Suas implementações são bem simples e intuitivas, mas sem deixarem de ser otimizadas e poderosas, buscando facilitar o desenvolvimento de projetos de reforço de alta qualidade.

Antes de começar a tarefa, é importante acessar e se familiarizar com o tutorial da biblioteca disponível neste repositório! Depois de rodar o guia, você já estará capaz de completar este trabalho.

## Escolha do Ambiente

Antes de analisar o possíveis algoritmos, o primeiro passo é escolher qual ambiente você quer resolver! Para esta tarefa, separamos dois possíveis ambientes diferentes, em ordem de dificuldade, que você poderá escolher: **CartPole** e **Pendulum**. Lembrando que, quanto mais difícil um ambiente, mais demorado será o treinamento.

A seguir, estão as descrições de cada um deles:

<h2 align="center">CartPole</h2>
<img src="https://bytepawn.com/images/cartpole.gif" width=50% />

**CartPole** é o ambiente de Aprendizado por Reforço mais comum do Gym, no qual deve-se balancear um pêndulo invertido conectado a um carrinho, somente controlando os movimentos do carrinho.

### Características do Ambiente

O **Espaço de Observação** do CartPole é definido por 4 informações:

<br>

|     | Informação                         | Min     | Max    |
| :-- | :--------------------------------- | :-----: | :----: |
| 0   | Posição do Carrinho                | -4.8    | 4.8    |
| 1   | Velocidade do Carrinho             | -Inf    | Inf    |
| 2   | Ângulo da Barra                    | -24 deg | 24 deg |
| 3   | Velocidade na Extremidade da Barra | -Inf    | Inf    |

<br>

A posição do carrinho vai de -4.8 a 4.8, mas ele perde o episódio caso saia dos limites de -2.4 e 2.4. Da mesma forma, o ângulo da barra vai de -24° a 24°, porém o episódio acaba caso a barra saia dos limites de -12° e 12°.

Já o **Espaço de Ação** é composto por duas ações únicas: mover o carrinho para a **esquerda** ou para a **direita**.

Quando queremos mover o carrinho para a esquerda, fazemos um `env.step(0)`; quando queremos movê-lo para a direita, enviamos um `env.step(1)`

| Ação | Significado           |
| :--- | :-------------------- |
| 0    | Mover para a esquerda |
| 1    | Mover para a direito  |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, que é igual a +1 para cada instante que passa sem o agente perder. Assim, o CartPole é incentivado a sobreviver por mais tempo.

Para testar o modelo, vamos precisar de 2 bibliotecas: **gym** (para inicialização dos ambientes) e **stable_baselines3** (para inicialização e avaliação dos modelos):

In [1]:
import gym 
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

# Definindo ambiente
env = gym.make("CartPole-v1")

# Definindo modelo
model = PPO("MlpPolicy", env, seed=1, verbose=1)

# Avaliando o agente
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Recompensa Média: 8.60 +/- 0.8


c:\Users\caste\Envs\main\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


<h2 align="center">Pendulum</h2>

<img src="https://www.gymlibrary.dev/_images/pendulum.gif" width=30% />

**Pendulum** é um ambiente do Gym que simula um pêndulo pendurado por um ponto tentando se balancear de cabeça para baixo. O agente deve um torque no pêndulo de forma que ele se levante e fique parado em pé.

### Características do Ambiente

O **Espaço de Observação** do ambiente é definido por 8 informações.

| Estado    | Informação                                     |
| :-------- | :--------------------------------------------- |
| 0         | Posição no eixo _x_ da ponta do pêndulo        |
| 1         | Posição no eixo _y_ da ponta do pêndulo        |
| 2         | Velocidade angular do pêndulo                  |

Já o **Espaço de Ação** é um espaço **contínuo** do torque aplicado no pêndulo.

| Ação | Significado     | Intervalo   |
| :--- | :-------------- | :---------- |
| 0    | Torque          | $-2$ a $+2$ |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, que segue a seguinte equação:

$$r = -(\theta{}^2 + 0.1 * \dot{\theta}^2 + 0.001 * \tau{})$$

Desta forma, temos que a recompensa é menor quando o ângulo do pêndulo é menor (mais em pé), quando a velocidade angular é baixa, e quando usamos pouco torque.

Por ser um ambiente com espaço de ação contínuo, os algoritmos que podemos usar serão diferentes, e o treinamento pode ser mais demorado.

### Ambiente

Para criar o ambiente, basta rodar a linha de código a seguir:

In [2]:
env = gym.make("Pendulum-v1")

### Exercício 1 - Testando Modelos

Caro piloto, agora que você conhece esses dois ambientes, é hora de brincar com eles. Você deverá testar diferentes algoritmos (a seu critério), e ver sua recompensa média. Para ver quais as limitações dos modelos, veja esse [link](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html). Abaixo, criamos uma função que será útil para comparar os modelos posteriormente.

In [3]:
def getValues(model, env, n_episodes, info_dict):
    model_name = str(model.__class__).split(".")[-1][:-2]

    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_episodes, deterministic=True)
    
    info_dict[model_name] = {}
    info_dict[model_name]["mean_reward"] = mean_reward
    info_dict[model_name]["std_reward"] = std_reward
    
    return info_dict

algorithms_dict = {}

#### Treinando e Avaliando seu próprio modelo

Primeiramente, agora você deve decidir em qual ambiente você deseja treinar seu agente. Para isto, basta tirar o comentário da linha referente ao ambiente escolhido:

In [4]:
# env_name = "CartPole-v1"
env_name = "Pendulum-v1"

Em seguida, você está livre para testar diferentes algoritmos para seu ambiente!

Como o ambiente escolhido foi o `Pendulum-v1`, usaremos apenas algoritmos que suportem espaços de ações contínuos, o que é equivalente a: `action_space = Box`. Além disso, como trabalharei com Single Process, testarei os algoritmos indicados [aqui](https://stable-baselines.readthedocs.io/en/master/guide/rl_tips.html), ou seja, `TD3` e `SAC`. Por fim, vou implementar alguns modelos usando tais algoritmos e comparar os resultados encontrados com o resultado obtido para um modelo com hiperparâmetros já tunados, disponíveis no [RL zoo](https://github.com/DLR-RM/rl-baselines3-zoo).

In [5]:
# Modelo de TD3 que usa hiperparâmetros já tunados do RL zoo
from stable_baselines3 import TD3 # Importe o modelo do stable_baselines3
from stable_baselines3.common.noise import NormalActionNoise
import numpy as np

# Definindo o ambiente
env = gym.make(env_name)

policy_kwargs = {'net_arch':[400,300]}
action_noise = NormalActionNoise(np.zeros(1), 0.1*np.ones(1))
model = TD3(policy='MlpPolicy', env=env, seed=1, verbose=1, buffer_size=int(2e5), learning_starts=int(1e4), gamma=0.98, policy_kwargs=policy_kwargs,
            action_noise=action_noise) # Defina o modelo
model.learn(total_timesteps= 2e4) # Treine o modelo
n_episodes = 20 # Defina o número de episódios

# Avaliando o agente e guardando o desempenho no dicionário
algorithms_dict = getValues(model, env, n_episodes, algorithms_dict)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.45e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 3137      |
|    time_elapsed    | 0         |
|    total_timesteps | 800       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.38e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 3361      |
|    time_elapsed    | 0         |
|    total_timesteps | 1600      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.28e+03 |
| time/              |           |
|    episodes        | 12      

In [7]:
# Modelo de SAC que usa hiperparâmetros já tunados do RL zoo
from stable_baselines3 import SAC # Importe o modelo do stable_baselines3

model = SAC(policy='MlpPolicy', env=env, seed=1, verbose=1, learning_rate=1e-3) # Defina o modelo
model.learn(total_timesteps= 2e4) # Treine o modelo
n_episodes = 20 # Defina o número de episódios

# Avaliando o agente e guardando o desempenho no dicionário
algorithms_dict = getValues(model, env, n_episodes, algorithms_dict)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.57e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 54        |
|    time_elapsed    | 14        |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 28.1      |
|    critic_loss     | 0.0398    |
|    ent_coef        | 0.5       |
|    ent_coef_loss   | -1.09     |
|    learning_rate   | 0.001     |
|    n_updates       | 699       |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -1.5e+03 |
| time/              |          |
|    episodes        | 8        |
|    fps             | 47       |
|    time_elapsed    | 33       |
|    total_timesteps | 1600     |
| tra

c:\Users\caste\Envs\main\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [13]:
# Resultado dos modelos com hiperparâmetros tunados
algorithms_dict = dict(zip(['TD3 ideal', 'SAC ideal'], list(algorithms_dict.values())))
algorithms_dict

{'TD3 ideal': {'mean_reward': -147.04256373576936,
  'std_reward': 114.15893494822676},
 'SAC ideal': {'mean_reward': -137.0330131453462,
  'std_reward': 98.30879141029631}}

Agora que temos os resultados ótimos, vou formular alguns modelos, tanto do `TD3` quanto do `SAC`, variando apenas alguns hiperparâmetros e deixando os outros com seu valor default, por fim, considerarei, apenas, o modelo com maior recompensa média. Os hiperparâmetros que irei variar para ambos algoritmos são:
- `gama`;
- `learning_rate`;
- `action_noise`(apenas para o `TD3`).

In [29]:
# Modelos de TD3
action_noise = [NormalActionNoise(np.zeros(1), 0.1*np.ones(1)), None]
gamma = [0.95, 0.9]
learning_rate = [3e-4, 6e-4,]
max_mean_reward = float('-inf')

for ac in action_noise:
    for g in gamma:
        for lr in learning_rate:
            model = TD3(policy='MlpPolicy', env=env, seed=1, verbose=1, learning_rate=lr, gamma=g, action_noise=ac) # Defina o modelo
            model.learn(total_timesteps= 5e3) # Treine o modelo
            n_episodes = 20 # Defina o número de episódios
            # Avaliando o agente e guardando o desempenho no dicionário
            mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_episodes, deterministic=True)
            if mean_reward > max_mean_reward:
                max_mean_reward = mean_reward
                max_std_reward = std_reward
                opt_gamma = g
                opt_lr = lr
                opt_ac = ac

algorithms_dict['TD3 teste'] = {'mean_reward': max_mean_reward, 'std_reward': max_std_reward}

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.58e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 95        |
|    time_elapsed    | 8         |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 15.2      |
|    critic_loss     | 0.295     |
|    learning_rate   | 0.0003    |
|    n_updates       | 600       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.55e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 78        |
|    time_elapsed    | 20        |
|    total_timesteps | 1600      |
| train/             |           |
|    actor_loss      | 26.1    

In [32]:
print('Melhor conjunto de hiperparâmetros encontrado para o TD3\naction_noise obtido: ' + str(opt_ac))
print(f'gamma obtido: {opt_gamma}')
print(f'learning_rate obtido: {opt_lr}')

Melhor conjunto de hiperparâmetros encontrado
action_noise obtido: None
gamma obtido: 0.9
learning_rate obtido: 0.0006


In [36]:
# Modelos de SAC
gamma = [0.95, 0.9]
learning_rate = [3e-4, 6e-4,]
max_mean_reward = float('-inf')

for g in gamma:
    for lr in learning_rate:
        model = SAC(policy='MlpPolicy', env=env, seed=1, verbose=1, learning_rate=lr, gamma=g) # Defina o modelo
        model.learn(total_timesteps= 1e4) # Treine o modelo
        n_episodes = 20 # Defina o número de episódios
        # Avaliando o agente e guardando o desempenho no dicionário
        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_episodes, deterministic=True)
        if mean_reward > max_mean_reward:
            max_mean_reward = mean_reward
            max_std_reward = std_reward
            opt_gamma = g
            opt_lr = lr

algorithms_dict['SAC teste'] = {'mean_reward': max_mean_reward, 'std_reward': max_std_reward}

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.52e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 48        |
|    time_elapsed    | 16        |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 24.1      |
|    critic_loss     | 0.2       |
|    ent_coef        | 0.812     |
|    ent_coef_loss   | -0.338    |
|    learning_rate   | 0.0003    |
|    n_updates       | 699       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.54e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 44        |
|    time_elapsed    | 36        |
|    total_timesteps | 1600    

c:\Users\caste\Envs\main\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.55e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 49        |
|    time_elapsed    | 16        |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 25.5      |
|    critic_loss     | 0.0885    |
|    ent_coef        | 0.659     |
|    ent_coef_loss   | -0.673    |
|    learning_rate   | 0.0006    |
|    n_updates       | 699       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.52e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 44        |
|    time_elapsed    | 35        |
|    total_timesteps | 1600    

In [37]:
print(f'Melhor conjunto de hiperparâmetros encontrado para o SAC\ngamma obtido: {opt_gamma}')
print(f'learning_rate obtido: {opt_lr}')

Melhor conjunto de hiperparâmetros encontrado para o SAC
gamma obtido: 0.95
learning_rate obtido: 0.0003


A seguir, detalhe um pouco mais quais foram os algoritmos testados bem como a performance obtida por cada um.

Este detalhamento pode ser feito por meio de um ou mais gráficos mostrando o desempenho dos modelos, ou simplesmente por texto.

In [39]:
### Espaço para o Piloto criar gráficos ou textos para mostrar os diferentes resultados entre modelos
algorithms_dict

{'TD3 ideal': {'mean_reward': -147.04256373576936,
  'std_reward': 114.15893494822676},
 'SAC ideal': {'mean_reward': -137.0330131453462,
  'std_reward': 98.30879141029631},
 'TD3 teste': {'mean_reward': -393.7203647630289,
  'std_reward': 100.32272172196895},
 'SAC teste': {'mean_reward': -155.9544133479707,
  'std_reward': 78.92418941143069}}

### Escolha do Algoritmo

Após testar e analisar diversos algoritmos diferentes, qual foi o escolhido?

_Pergunta Extra:_ você usou algum critério para escolher quais algoritmos seriam testados?

Analisando os resultados obtidos, a escolha mais adequada é do `SAC`, visto que foi o algoritmo que teve a maior recompensa média, tanto dentre os modelos considerados ideais, quanto para os modelos que foram formulados e tunados aqui.